##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Melhor engenharia de ML com o ML Metadata


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/mlmd/mlmd_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  
</table>

Suponha um cenário em que você configura um pipeline de ML de produção para classificar pinguins. O pipeline ingere seus dados de treinamento, treina e avalia um modelo e o envia para produção.

No entanto, quando você tenta usar esse modelo posteriormente com um dataset maior que contém diferentes tipos de pinguins, você observa que seu modelo não se comporta conforme o esperado e começa a classificar as espécies incorretamente.

Neste ponto, você estará interessado em saber:

- Qual é a maneira mais eficiente de depurar o modelo quando o único artefato disponível é o modelo em produção?
- Qual dataset de treinamento foi usado para treinar o modelo?
- Qual execução de treinamento levou a esse modelo incorreto?
- Onde estão os resultados da avaliação do modelo?
- Por onde começar a depuração?

O [ML Metadata (MLMD)](https://github.com/google/ml-metadata) é uma biblioteca que aproveita os metadados associados aos modelos de ML para ajudá-lo a responder a essas perguntas e muito mais. Uma analogia útil é pensar nesses metadados como o equivalente ao registro no desenvolvimento de software. O MLMD permite rastrear de forma confiável os artefatos e a linhagem associados aos vários componentes do seu pipeline de ML.

Neste tutorial, você vai configurar um pipeline TFX para criar um modelo que classifica os pinguins em três espécies com base na massa corporal e no comprimento e profundidade de seus culmens e no comprimento de suas nadadeiras. Em seguida, você usará o MLMD para rastrear a linhagem dos componentes do pipeline.

## Pipelines TFX no Colab

O Colab é um ambiente de desenvolvimento leve que difere significativamente de um ambiente de produção. Na produção, você pode ter vários componentes de pipeline, como ingestão de dados, transformação, treinamento de modelo, históricos de execução, etc., distribuídos por múltiplos sistemas. Para este tutorial, você deve estar ciente de que existem diferenças significativas na orquestração e no armazenamento de metadados: tudo é feito localmente no Colab. Saiba mais sobre o TFX no Colab [aqui](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras#background).


## Configuração

Primeiro, instalamos e importamos os pacotes necessários, configuramos caminhos e baixamos os dados.

### Atualize o Pip

Para evitar a atualização do Pip num sistema ao executar localmente, garanta que estamos executando no Colab. Os sistemas locais podem, claro, ser atualizados separadamente.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instale e importe o TFX

In [ ]:
 !pip install -q tfx

### Importe os pacotes

#### Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime clicando no botão "RESTART RUNTIME" acima ou usando o menu "Runtime &gt; Restart runtime ...". Isso é necessário devido à maneira como o Colab carrega os pacotes.

In [ ]:
import os
import tempfile
import urllib
import pandas as pd

import tensorflow_model_analysis as tfma
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

Verifique as versões do TFX e do MLMD.

In [ ]:
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import ml_metadata as mlmd
print('MLMD version: {}'.format(mlmd.__version__))

## Download do dataset

Nesta colab, usamos o [dataset Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) que pode ser encontrado no [Github](https://github.com/allisonhorst/palmerpenguins). Processamos o dataset omitindo quaisquer registros incompletos, eliminando as colunas `island` e `sex` e convertendo rótulos para `int32`. O dataset contém 334 registros da massa corporal e do comprimento e profundidade dos culmens dos pinguins e do comprimento de suas nadadeiras. Você usará esses dados para classificar os pinguins em uma das três espécies.

In [ ]:
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_root = tempfile.mkdtemp(prefix='tfx-data')
_data_filepath = os.path.join(_data_root, "penguins_processed.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

## Crie um InteractiveContext

Para executar componentes do TFX interativamente neste notebook, crie um `InteractiveContext`. O `InteractiveContext` usa um diretório temporário com uma instância de banco de dados MLMD efêmera. Observe que as chamadas para `InteractiveContext` não funcionam fora do ambiente Colab.

Em geral, é uma boa prática agrupar execuções de um pipeline similares em um `Context` .

In [ ]:
interactive_context = InteractiveContext()

## Construa o pipeline TFX

Um pipeline TFX consiste em vários componentes que executam diferentes aspectos do workflow de ML. Neste notebook, você criará e executará os componentes `ExampleGen`, `StatisticsGen`, `SchemaGen` e `Trainer` e usará os componentes `Evaluator` e `Pusher` para avaliar e enviar por push o modelo treinado.

Confira o [tutorial de componentes](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) para mais informações sobre os componentes do pipeline TFX.

Observação: construir um pipeline TFX configurando os componentes individuais envolve escrever muito código boilerplate. Para os fins deste tutorial, não há problema se você não compreender totalmente cada linha de código na configuração do pipeline. 

### Instancie e execute o componente ExampleGen

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
interactive_context.run(example_gen)

### Instancie e execute o componente StatisticsGen

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
interactive_context.run(statistics_gen)

### Instancie e execute o componente SchemaGen

In [ ]:
infer_schema = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
interactive_context.run(infer_schema)

### Instancie e execute o componente Trainer


In [ ]:
# Define the module file for the Trainer component
trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {trainer_module_file}

# Define the training algorithm for the Trainer module file
import os
from typing import List, Text

import tensorflow as tf
from tensorflow import keras

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

# Features used for classification - culmen length and depth, flipper length,
# body mass, and species.

_LABEL_KEY = 'species'

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema, batch_size: int) -> tf.data.Dataset:
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY), schema).repeat()


def _build_keras_model():
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  d = keras.layers.Dense(8, activation='relu')(d)
  d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])
  return model


def run_fn(fn_args: tfx.components.FnArgs):
  schema = schema_pb2.Schema()
  tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema)
  train_dataset = _input_fn(
      fn_args.train_files, fn_args.data_accessor, schema, batch_size=10)
  eval_dataset = _input_fn(
      fn_args.eval_files, fn_args.data_accessor, schema, batch_size=10)
  model = _build_keras_model()
  model.fit(
      train_dataset,
      epochs=int(fn_args.train_steps / 20),
      steps_per_epoch=20,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)
  model.save(fn_args.serving_model_dir, save_format='tf')

Execute o componente `Trainer`.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(trainer_module_file),
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=50))
interactive_context.run(trainer)

### Avalie e faça o push do modelo

Use o componente `Evaluator` para avaliar e 'abençoar' o modelo antes de usar o componente `Pusher` para enviar o modelo via push para um diretório de serviço.

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(),
                                  'serving_model/penguins_classification')

In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='species', signature_name='serving_default')
    ],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='SparseCategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.6})))
        ])
    ],
    slicing_specs=[tfma.SlicingSpec()])

In [ ]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    schema=infer_schema.outputs['schema'],
    eval_config=eval_config)
interactive_context.run(evaluator)

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
interactive_context.run(pusher)

A execução do pipeline TFX preenche o banco de dados MLMD. Na próxima seção, você usará a API MLMD para consultar esse banco de dados em busca de informações de metadados.

## Consulte o banco de dados MLMD

O banco de dados MLMD armazena três tipos de metadados:

- Metadados sobre o pipeline e informações de linhagem associadas aos componentes do pipeline
- Metadados sobre artefatos que foram gerados durante a execução do pipeline
- Metadados sobre as execuções do pipeline

Um pipeline típico de ambiente de produção atende a múltiplos modelos à medida que novos dados chegam. Ao encontrar resultados incorretos em modelos servidos, você pode consultar o banco de dados MLMD para isolar os modelos errados. Você pode então rastrear a linhagem dos componentes do pipeline que correspondem a esses modelos para depurar seus modelos

Configure o armazenamento de metadados (MD) com o `InteractiveContext` definido anteriormente para consultar o banco de dados MLMD.

In [ ]:
connection_config = interactive_context.metadata_connection_config
store = mlmd.MetadataStore(connection_config)

# All TFX artifacts are stored in the base directory
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

Crie algumas funções helper para visualizar os dados do armazenamento MD.

In [ ]:
def display_types(types):
  # Helper function to render dataframes for the artifact and execution types
  table = {'id': [], 'name': []}
  for a_type in types:
    table['id'].append(a_type.id)
    table['name'].append(a_type.name)
  return pd.DataFrame(data=table)

In [ ]:
def display_artifacts(store, artifacts):
  # Helper function to render dataframes for the input artifacts
  table = {'artifact id': [], 'type': [], 'uri': []}
  for a in artifacts:
    table['artifact id'].append(a.id)
    artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
    table['type'].append(artifact_type.name)
    table['uri'].append(a.uri.replace(base_dir, './'))
  return pd.DataFrame(data=table)

In [ ]:
def display_properties(store, node):
  # Helper function to render dataframes for artifact and execution properties
  table = {'property': [], 'value': []}
  for k, v in node.properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  for k, v in node.custom_properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  return pd.DataFrame(data=table)

Primeiro, consulte o armazenamento MD para obter uma lista de todos os seus `ArtifactTypes` armazenados.

In [ ]:
display_types(store.get_artifact_types())

Em seguida, consulte todos os artefatos `PushedModel`.

In [ ]:
pushed_models = store.get_artifacts_by_type("PushedModel")
display_artifacts(store, pushed_models)

Consulte o armazenamento MD para obter o modelo que foi enviado mais recentemente. Este tutorial tem apenas um modelo enviado. 

In [ ]:
pushed_model = pushed_models[-1]
display_properties(store, pushed_model)

Uma das primeiras etapas na depuração de um modelo enviado é observar qual modelo treinado foi enviado e quais dados de treinamento são usados ​​para treinar esse modelo.

O MLMD fornece APIs de travessia para percorrer o gráfico de proveniência, que você pode usar para analisar a proveniência do modelo. 

In [ ]:
def get_one_hop_parent_artifacts(store, artifacts):
  # Get a list of artifacts within a 1-hop of the artifacts of interest
  artifact_ids = [artifact.id for artifact in artifacts]
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids(artifact_ids)
      if event.type == mlmd.proto.Event.OUTPUT)
  artifacts_ids = set(
      event.artifact_id
      for event in store.get_events_by_execution_ids(executions_ids)
      if event.type == mlmd.proto.Event.INPUT)
  return [artifact for artifact in store.get_artifacts_by_id(artifacts_ids)]

Consulte os artefatos pai para o modelo enviado.

In [ ]:
parent_artifacts = get_one_hop_parent_artifacts(store, [pushed_model])
display_artifacts(store, parent_artifacts)

Consulte as propriedades do modelo.

In [ ]:
exported_model = parent_artifacts[0]
display_properties(store, exported_model)

Consulte os artefatos upstream para o modelo.

In [ ]:
model_parents = get_one_hop_parent_artifacts(store, [exported_model])
display_artifacts(store, model_parents)

Obtenha os dados de treinamento com os quais o modelo foi treinado.

In [ ]:
used_data = model_parents[0]
display_properties(store, used_data)

Agora que você tem os dados de treinamento com os quais o modelo foi treinado, consulte o banco de dados novamente para encontrar a etapa de treinamento (execução). Consulte o armazenamento MD para obter uma lista dos tipos de execução registrados.

In [ ]:
display_types(store.get_execution_types())

A etapa de treinamento é o `ExecutionType` denominado `tfx.components.trainer.component.Trainer`. Percorra o armazenamento MD para obter a execução do treinador que corresponde ao modelo enviado.

In [ ]:
def find_producer_execution(store, artifact):
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids([artifact.id])
      if event.type == mlmd.proto.Event.OUTPUT)
  return store.get_executions_by_id(executions_ids)[0]

trainer = find_producer_execution(store, exported_model)
display_properties(store, trainer)

## Resumo

Neste tutorial, você aprendeu como aproveitar o MLMD para rastrear a linhagem dos componentes do pipeline do TFX e resolver problemas.

Para saber mais sobre como usar o MLMD, confira estes recursos adicionais:

- [Documentação da API MLMD](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd)
- [Guia do MLMD](https://www.tensorflow.org/tfx/guide/mlmd)